# FK-RFdiffusion: Binder Design Tutorial

This notebook demonstrates how to design protein binders using Feynman-Kac guided RFdiffusion.

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/ErikHartman/FK-RFdiffusion/blob/main/examples/binder_design.ipynb)

**What you'll learn:**
- Set up the environment in Colab
- Design binders optimized for binding affinity
- Visualize and analyze results

## 1. Environment Setup

First, let's install all required dependencies. This will take ~5-10 minutes.

In [ ]:
# Check GPU availability
!nvidia-smi

In [ ]:
# Install PyTorch with CUDA support
!pip install -q torch==2.4.0+cu121 torchvision==0.19.0+cu121 torchaudio==2.4.0+cu121 \
  --extra-index-url https://download.pytorch.org/whl/cu121

In [ ]:
# Install DGL with CUDA support
!pip install -q --no-cache-dir "dgl==2.1.0+cu121" \
  -f https://data.dgl.ai/wheels/cu121/repo.html

In [ ]:
# Install other dependencies
!pip install -q hydra-core==1.3.2 omegaconf==2.3.0 tqdm biopython pandas torchdata>=0.7,<0.8
!pip install -q pydssp

## 2. Clone FK-RFdiffusion and Dependencies

In [ ]:
# Clone the repository with submodules
!git clone --recursive https://github.com/ErikHartman/FK-RFdiffusion.git
%cd FK-RFdiffusion

In [ ]:
# Install RFdiffusion
%cd externals/RFdiffusion
!pip install -q -e . --no-deps

# Install SE(3) Transformer
%cd env/SE3Transformer
!pip install -q -r requirements.txt
!pip install -q .
%cd ../../../

In [ ]:
# Install ProteinMPNN (for sequence design in reward functions)
%cd externals/ProteinMPNN
!pip install -q -e .
%cd ../..

## 3. Download RFdiffusion Model Weights

In [ ]:
# Download the Complex/Binder checkpoint
!mkdir -p externals/RFdiffusion/models
!wget -q http://files.ipd.uw.edu/pub/RFdiffusion/e29311f6f1bf1af907f9ef9f44b8328b/Complex_base_ckpt.pt \
  -O externals/RFdiffusion/models/Complex_base_ckpt.pt

print("✓ Model checkpoint downloaded")

## 4. Prepare Target Structure

For this example, we'll use a PDB structure. You can either:
- Upload your own PDB file
- Download from PDB database

In [ ]:
# Example: Download a structure from PDB (insulin receptor)
!mkdir -p data
!wget -q https://files.rcsb.org/download/1IRK.pdb -O data/target.pdb

target_structure = "data/target.pdb"
print(f"Target structure: {target_structure}")

## 5. Run FK-Guided Binder Design

Now let's design binders optimized for binding affinity using Feynman-Kac guidance.

In [ ]:
import sys
sys.path.insert(0, '.')

from fk_rfdiffusion.run_inference_guided import run_feynman_kac_design

### Design a 20-residue binder to chain A, residues 1-50

In [ ]:
run_feynman_kac_design(
    contigs=["A1-50/0 20"],              # Target residues A1-50, then 20-residue binder
    target_structure=target_structure,    # Your target protein
    reward_function="interface_dG",       # Optimize binding energy
    num_designs=5,                        # Generate 5 designs
    n_particles=15,                       # Use 15 parallel particles
    resampling_frequency=5,               # Resample every 5 steps
    guidance_start_timestep=30,           # Start guiding at timestep 30
    potential_mode="difference",          # Use difference-based potential
    output_prefix="./outputs/binder",     # Output directory
    checkpoint="base"                     # Use base checkpoint
)

### Variable-length binder design

Design binders with variable length (15-25 residues) to explore different binding modes.

In [ ]:
run_feynman_kac_design(
    contigs=["A1-50/0 15-25"],            # Variable length: 15-25 residues
    target_structure=target_structure,
    reward_function="interface_dG",
    num_designs=1,                        # 1 design per run
    n_runs=10,                            # 10 runs with different lengths
    n_particles=20,
    resampling_frequency=5,
    guidance_start_timestep=30,
    output_prefix="./outputs/var_binder"
)

## 6. Visualize Results

Let's visualize the generated binders using py3Dmol.

In [ ]:
!pip install -q py3Dmol

In [ ]:
import py3Dmol
import glob

# Get the first designed structure
pdb_files = sorted(glob.glob("outputs/binder*.pdb"))

if pdb_files:
    pdb_file = pdb_files[0]
    
    # Read PDB file
    with open(pdb_file, 'r') as f:
        pdb_data = f.read()
    
    # Create viewer
    view = py3Dmol.view(width=800, height=600)
    view.addModel(pdb_data, 'pdb')
    
    # Style: target in gray, binder in blue
    view.setStyle({'chain': 'A'}, {'cartoon': {'color': 'gray'}})
    view.setStyle({'chain': 'B'}, {'cartoon': {'color': 'blue'}})
    
    view.zoomTo()
    view.show()
else:
    print("No PDB files found. Make sure the design completed successfully.")

## 7. Analyze Designs

Compare trajectories and final structures.

In [ ]:
# List all generated designs
!ls -lh outputs/

In [ ]:
# View multiple designs side-by-side
pdb_files = sorted(glob.glob("outputs/binder*.pdb"))[:3]  # First 3 designs

for i, pdb_file in enumerate(pdb_files):
    print(f"\n=== Design {i+1}: {pdb_file} ===")
    
    with open(pdb_file, 'r') as f:
        pdb_data = f.read()
    
    view = py3Dmol.view(width=400, height=300)
    view.addModel(pdb_data, 'pdb')
    view.setStyle({'chain': 'A'}, {'cartoon': {'color': 'gray'}})
    view.setStyle({'chain': 'B'}, {'cartoon': {'color': 'spectrum'}})
    view.zoomTo()
    view.show()

## 8. Export Designs

Download your designed binders to your local machine.

In [ ]:
# Zip all outputs
!zip -r binder_designs.zip outputs/

# Download in Colab
from google.colab import files
files.download('binder_designs.zip')

## Next Steps

- **Optimize further**: Try different values of `n_particles`, `guidance_start_timestep`, or `tau`
- **Use hotspots**: Specify `hotspot_res=["A10", "A15"]` to focus on specific residues
- **Multiple sequences**: Set `n_sequences=5` to average over multiple sequence designs
- **Experimental validation**: Test top candidates in the lab!